In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg

In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
####### inputs
N, k = 20, 3  # toy size
# random SPD matrices K, M
A = np.random.randn(N, N)
K = A.T @ A + np.eye(N)   # make SPD
B = np.random.randn(N, N)
M = B.T @ B + np.eye(N)   # make SPD

K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)

# reference solution
w_all, V_all = linalg.eigh(K.cpu().numpy(), M.cpu().numpy())
print("Reference eigs:", np.round(w_all[:k], 6))


X = torch.randn(N, 3, device=device)


Reference eigs: [0.037543 0.084728 0.098262]


In [13]:
# Build the neural network that maps coordinates -> k outputs per node
class MLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=k, hidden=[64,64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last, h, dtype=torch.double))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, out_dim, dtype=torch.double))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# Instantiate model and optimizer
model = MLP().to(device)
# initialize final layer small
for name, p in model.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
max_epochs = 20_000
print_every = 1_000
loss_history = []

In [15]:
for epoch in range(1, max_epochs+1):
    optimizer.zero_grad()
    U = model(X)  # N x k

    # losses
    B = U.T @ (M @ U)        # k x k
    orth_loss = torch.norm(B - torch.eye(k, device=device))**2
    eig_loss = torch.trace(U.T @ (K @ U))

    loss = eig_loss + orth_loss
    loss.backward()
    optimizer.step()

    if epoch % print_every == 0 or epoch == 1:
        approx_vals = torch.diag(U.T @ (K @ U)).detach().cpu().numpy()
        print(f"Epoch {epoch}, total loss={loss.item():.6f}, approx eigs={np.round(approx_vals,6)}")


Epoch 1, total loss=5466.097885, approx eigs=[44.427397 79.294931 14.539985]
Epoch 1000, total loss=0.526866, approx eigs=[0.171939 0.129668 0.196167]
Epoch 2000, total loss=0.321120, approx eigs=[0.088562 0.100317 0.117696]
Epoch 3000, total loss=0.253002, approx eigs=[0.063946 0.088924 0.092041]
Epoch 4000, total loss=0.225534, approx eigs=[0.053354 0.078629 0.087767]
Epoch 5000, total loss=0.217493, approx eigs=[0.052289 0.073736 0.086535]
Epoch 6000, total loss=0.216043, approx eigs=[0.052555 0.072592 0.086272]
Epoch 7000, total loss=0.215973, approx eigs=[0.052647 0.072499 0.086261]
Epoch 8000, total loss=0.215972, approx eigs=[0.052656 0.072498 0.086257]
Epoch 9000, total loss=0.215975, approx eigs=[0.052659 0.072471 0.086223]
Epoch 10000, total loss=0.216033, approx eigs=[0.052759 0.072441 0.086297]
Epoch 11000, total loss=0.215972, approx eigs=[0.052805 0.072447 0.086164]
Epoch 12000, total loss=0.216176, approx eigs=[0.05302  0.072477 0.086047]
Epoch 13000, total loss=0.217194

In [16]:
# ==== Final result ====
with torch.no_grad():
    U_final = model(X)
    UKU = U_final.T @ (K @ U_final)
    mu, _ = torch.linalg.eigh(UKU)
    print("\nLearned Ritz values:", np.round(mu.cpu().numpy(), 6))
    print("Reference eigs:     ", np.round(w_all[:k], 6))


Learned Ritz values: [0.040365 0.082175 0.093786]
Reference eigs:      [0.037543 0.084728 0.098262]
